In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from transformers import BertTokenizer, TFBertForSequenceClassification


2023-06-23 01:35:27.687677: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv("amazon_reviews_3.csv")

In [7]:
train_texts = df['REVIEW_TEXT'].values
train_labels = df['LABEL_ENCODED'].values

In [5]:
train_labels = keras.utils.to_categorical(train_labels)

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer.batch_encode_plus(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128  # Adjust as needed
)

In [13]:
dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(100)

In [14]:
train_size = int(0.8 * len(dataset))
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [15]:
train_dataset = train_dataset.batch(32)  # Adjust batch size as needed
val_dataset = val_dataset.batch(32)

In [16]:
# Load the pre-trained BERT model with a classification head
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Train the model
model.fit(
    train_dataset,
    epochs=5,  # Adjust the number of epochs as needed
    validation_data = val_dataset
)

Epoch 1/5


2023-06-23 01:45:03.929209: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [21000]
	 [[{{node Placeholder/_3}}]]
2023-06-23 01:45:03.929940: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype int64 and shape [21000]
	 [[{{node Placeholder/_3}}]]


ValueError: in user code:

    File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/transformers/modeling_tf_utils.py", line 1692, in train_step
        loss = self.compiled_loss(y, y_pred, sample_weight, regularization_losses=self.losses)
    File "/Users/kpandey/anaconda3/lib/python3.10/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/Users/kpandey/anaconda3/lib/python3.10/site-

In [ ]:
# Assuming you have test_texts containing new review texts
test_encodings = tokenizer.batch_encode_plus(
    test_texts,
    truncation=True,
    padding=True,
    max_length=128  # Adjust as needed
)

test_dataset = tf.data.Dataset.from_tensor_slices(
    dict(test_encodings)
).batch(32)  # Adjust batch size as needed

predictions = model.predict(test_dataset)